In [5]:
!pip install pyreadstat==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import pyreadstat
import pandas as pd
from collections import defaultdict
import math
from scipy.stats import chisquare
import plotly.express as px

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Используем данные 2020 года, ибо в более ранних нет вопроса про дистанционную работу

In [9]:
fpath = 'drive/MyDrive/HHOLD_OSN_2020.sav' # Источник данных: https://gks.ru .

In [11]:
df, meta = pyreadstat.read_file_multiprocessing(pyreadstat.read_sav, fpath, num_processes=16, encoding='cp1251') 
df.head()

,H00_02,H00_04,H00_06,H00_07,H00_08,H00_09,H00_10,H00_11,H00_12,H00_13,...,PM_DX_MES,PM_DX_SRED_MES,MALOIM,KVZV,SUPR_PAR,DX,V407,GL_DH_POL,GL_DH_VOZR,GL_DH_LET
0,46,2.0,37419.0,9.0,22.0,10.0,2020.0,13.0,0.0,13.0,...,20206.0,10103.0,0.0,575.713802,1.0,1.0,NaN,1.0,2.0,70.0
1,46,2.0,37702.0,9.0,22.0,10.0,2020.0,15.0,0.0,15.0,...,58140.0,14535.0,0.0,1025.493642,1.0,NaN,NaN,1.0,1.0,50.0
2,46,2.0,37703.0,9.0,23.0,10.0,2020.0,11.0,0.0,11.0,...,58140.0,14535.0,1.0,882.435870,1.0,NaN,NaN,1.0,1.0,30.0
3,46,2.0,37073.0,9.0,23.0,10.0,2020.0,17.0,0.0,17.0,...,10103.0,10103.0,0.0,525.487031,NaN,1.0,NaN,2.0,2.0,69.0
4,46,2.0,37074.0,9.0,24.0,10.0,2020.0,10.0,0.0,10.0,...,14941.0,14941.0,0.0,757.206120,NaN,NaN,NaN,1.0,1.0,56.0


Что такое частота - читаем туть (да, это сайт с оформлением из нулевых и смешной рекламой, но зато всё понятно): http://mathprofi.ru/matematicheskaya_statistika.html

In [12]:
columns_to_count_frequencies = [
    'H02_13_01_01', 'H02_13_02_01', 'H02_18', 'H02_221_01', 'H02_221_02', 'H02_221_03', 'H02_23', 'H02_27', 'H02_29', 'H02_350', 'H02_36_01_01', 'H02_36_04_01', 'H02_09_05', 'H02_09_06', 
    'H02_09_08', 'H02_10', 'H02_01', 'H02_03',
    ]

документация pandas - есть наглядный пример разницы между частотами и частостями (у них они называются relative frequencies)
https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html


In [15]:
# загоняем всё под функцию, чтобы не повторяться
def frequencuies_pandas(dataframe, columns_to_count_frequencies, factors, name):
    for column in columns_to_count_frequencies:
        try:
            question = meta.column_names_to_labels[column]
            print(question)
            frequencies = dataframe[column].value_counts(dropna=False)
            # value_counts возвращает Series, где value это индекс, а значения - частоты или частости
            frequencies_rel = dataframe[column].value_counts(normalize=True, dropna=False)

            tuples = [(name, 'Абс.'), (name, 'Отн.')]
            index = pd.MultiIndex.from_tuples(tuples)
            question_dict = {'Абс.': frequencies, 'Отн.':frequencies_rel}
            question_df = pd.DataFrame(question_dict)
            
            question_df.index = pd.Series(question_df.index).apply(lambda x: meta.value_labels[meta.variable_to_label[column]].get(x))
            fig = px.pie(question_df, values='Абс.', names=question_df.index, title=question)
            fig.show()
            total = {'Абс.': question_df['Абс.'].sum(), 'Отн.': question_df['Отн.'].sum()}
            new_row = pd.DataFrame(total, index=['Total'])
            question_df = pd.concat([question_df, new_row], axis=0)

            question_df.columns = index
            print(question_df)
            factors[question].append(question_df)
        except KeyError: # обработка ошибки: если нет метки spss для столбца, выругиваемся и переходим к следующему столбцу
            print(f"{column} отсутствует в meta")
        print('-------------------------------------------------------------------')

In [16]:
factors = defaultdict(list)
frequencuies_pandas(df, columns_to_count_frequencies, factors, 'Вся РФ')
print('===================================================================')

Наличие лифта


      Вся РФ          
        Абс.      Отн.
Нет    26587  0.443161
None   20905  0.348452
Да     12502  0.208388
Total  59994  1.000000
-------------------------------------------------------------------
Наличие мусоропровода


      Вся РФ          
        Абс.      Отн.
Нет    28051  0.467563
None   20905  0.348452
Да     11038  0.183985
Total  59994  1.000000
-------------------------------------------------------------------
Вид водопровода в доме


                                                   Вся РФ          
                                                     Абс.      Отн.
Централизованный водопровод                         51702  0.861786
Водопровод отсутствует                               3287  0.054789
Водопровод из индивидуальной артезианской скважины   2909  0.048488
Водопровод из колодца                                2079  0.034653
None                                                   17  0.000283
Total                                               59994  1.000000
-------------------------------------------------------------------
Наличие стиральной машины


                                         Вся РФ          
                                           Абс.      Отн.
Да и имеется и используется               58290  0.971597
Нет, не имеется, не можем себе позволить    635  0.010584
Да, имеется, но не используется             406  0.006767
Нет, не имею по другим причинам             331  0.005517
Нет, не имеется, нет необходимости          315  0.005251
None                                         17  0.000283
Total                                     59994  1.000000
-------------------------------------------------------------------
Наличие холодильника


...

               Вся РФ         
                 Абс.     Отн.
None            54195  0.90334
шум от соседей   5799  0.09666
Total           59994  1.00000
-------------------------------------------------------------------
Нарушение общественного порядка вблизи дома или в подъезде


                                                   Вся РФ          
                                                     Абс.      Отн.
None                                                57838  0.964063
нарушение общественного порядка вблизи дома или...   2156  0.035937
Total                                               59994  1.000000
-------------------------------------------------------------------
Оценка состояния жилого помещения


                   Вся РФ          
                     Абс.      Отн.
Удовлетворительное  27005  0.450128
Хорошее             26372  0.439577
Отличное             4039  0.067323
Плохое               2343  0.039054
Очень плохое          235  0.003917
Total               59994  1.000000
-------------------------------------------------------------------
Условия проживания домохозяйства


                                             Вся РФ          
                                               Абс.      Отн.
Отдельная квартира                            38786  0.646498
Индивидуальный (одноквартирный) дом           17711  0.295213
Часть индивидуального (одноквартирного) дома   3177  0.052955
Коммунальная квартира                           251  0.004184
Общежитие                                        53  0.000883
Другое                                           16  0.000267
Total                                         59994  1.000000
-------------------------------------------------------------------
Является ли основное жилье собственностью домохозяйства


                                                   Вся РФ          
                                                     Абс.      Отн.
Да, без использования ипотечных средств или ипо...  50710  0.845251
Нет                                                  6162  0.102710
Да, с использованием ипотечных средств (ипотека...   3053  0.050888
None                                                   69  0.001150
Total                                               59994  1.000000
-------------------------------------------------------------------


In [17]:
# и так же, как  в ноутбуке descriptive_statistics, выбираем городскую и сельскую местность
for name in df['R_1'].unique():
    selected = df[df['R_1']==name]
    name = meta.value_labels[meta.variable_to_label['R_1']].get(name)
    frequencuies_pandas(selected, columns_to_count_frequencies, factors, name)
    print('===================================================================')

Наличие лифта


      сельская местность          
                    Абс.      Отн.
None               13036  0.705870
Нет                 5331  0.288661
Да                   101  0.005469
Total              18468  1.000000
-------------------------------------------------------------------
Наличие мусоропровода


      сельская местность          
                    Абс.      Отн.
None               13036  0.705870
Нет                 5325  0.288337
Да                   107  0.005794
Total              18468  1.000000
-------------------------------------------------------------------
Вид водопровода в доме


                                                   сельская местность  \
                                                                 Абс.   
Централизованный водопровод                                     12813   
Водопровод отсутствует                                           2324   
Водопровод из индивидуальной артезианской скважины               1947   
Водопровод из колодца                                            1376   
None                                                                8   
Total                                                           18468   

                                                              
                                                        Отн.  
Централизованный водопровод                         0.693795  
Водопровод отсутствует                              0.125839  
Водопровод из индивидуальной артезианской скважины  0.105426  
Водопровод из колодца                               0.074507  
None                                 

                                         сельская местность          
                                                       Абс.      Отн.
Да и имеется и используется                           17516  0.948451
Нет, не имеется, не можем себе позволить                366  0.019818
Да, имеется, но не используется                         232  0.012562
Нет, не имею по другим причинам                         185  0.010017
Нет, не имеется, нет необходимости                      161  0.008718
None                                                      8  0.000433
Total                                                 18468  1.000000
-------------------------------------------------------------------
Наличие холодильника


...

               городская местность          
                              Абс.      Отн.
None                         36565  0.880533
шум от соседей                4961  0.119467
Total                        41526  1.000000
-------------------------------------------------------------------
Нарушение общественного порядка вблизи дома или в подъезде


                                                   городская местность  \
                                                                  Абс.   
None                                                             39662   
нарушение общественного порядка вблизи дома или...                1864   
Total                                                            41526   

                                                              
                                                        Отн.  
None                                                0.955112  
нарушение общественного порядка вблизи дома или...  0.044888  
Total                                               1.000000  
-------------------------------------------------------------------
Оценка состояния жилого помещения


                   городская местность          
                                  Абс.      Отн.
Хорошее                          19030  0.458267
Удовлетворительное               18072  0.435197
Отличное                          3013  0.072557
Плохое                            1274  0.030680
Очень плохое                       137  0.003299
Total                            41526  1.000000
-------------------------------------------------------------------
Условия проживания домохозяйства


                                             городская местность          
                                                            Абс.      Отн.
Отдельная квартира                                         33420  0.804797
Индивидуальный (одноквартирный) дом                         6859  0.165174
Часть индивидуального (одноквартирного) дома                1001  0.024105
Коммунальная квартира                                        198  0.004768
Общежитие                                                     40  0.000963
Другое                                                         8  0.000193
Total                                                      41526  1.000000
-------------------------------------------------------------------
Является ли основное жилье собственностью домохозяйства


                                                   городская местность  \
                                                                  Абс.   
Да, без использования ипотечных средств или ипо...               34672   
Нет                                                               4179   
Да, с использованием ипотечных средств (ипотека...                2627   
None                                                                48   
Total                                                            41526   

                                                              
                                                        Отн.  
Да, без использования ипотечных средств или ипо...  0.834947  
Нет                                                 0.100636  
Да, с использованием ипотечных средств (ипотека...  0.063262  
None                                                0.001156  
Total                                               1.000000  
----------------------------------------

In [18]:
yakutia = df[df['H00_02']=='98'] # маской выбираем по коду субъекта только Якутию, как делали в ноутбуке descriptive_statistics
#! 98 это строка, а не число. Если без кавычек, то ничего не найдется :)

In [19]:
yakutia.head()

,H00_02,H00_04,H00_06,H00_07,H00_08,H00_09,H00_10,H00_11,H00_12,H00_13,...,PM_DX_MES,PM_DX_SRED_MES,MALOIM,KVZV,SUPR_PAR,DX,V407,GL_DH_POL,GL_DH_VOZR,GL_DH_LET
999,98,1.0,70348.0,4.0,24.0,10.0,2020.0,15.0,40.0,16.0,...,18728.0,18728.000000,0.0,407.593312,NaN,NaN,NaN,2.0,1.0,50.0
1000,98,1.0,70364.0,4.0,21.0,10.0,2020.0,15.0,36.0,16.0,...,18728.0,18728.000000,0.0,407.593312,NaN,NaN,NaN,2.0,1.0,50.0
1001,98,1.0,70362.0,4.0,25.0,10.0,2020.0,12.0,30.0,13.0,...,18728.0,18728.000000,1.0,407.593312,NaN,NaN,NaN,2.0,1.0,25.0
1002,98,1.0,70465.0,4.0,24.0,10.0,2020.0,17.0,35.0,18.0,...,37456.0,18728.000000,0.0,1117.087036,1.0,NaN,NaN,1.0,1.0,52.0
1003,98,1.0,70577.0,4.0,1.0,11.0,2020.0,16.0,0.0,16.0,...,55846.0,18615.333333,1.0,954.453765,NaN,NaN,NaN,2.0,1.0,38.0


In [20]:
# и так же, как  в ноутбуке descriptive_statistics, выбираем городскую и сельскую местность
for name in yakutia['R_1'].unique():
    print(name)
    selected = yakutia[yakutia['R_1']==name]
    name = meta.value_labels[meta.variable_to_label['R_1']].get(name)
    
    print(selected)
    frequencuies_pandas(selected, columns_to_count_frequencies, factors, f'Якутия {name}')
    print('===================================================================')

1.0
      H00_02  H00_04   H00_06  H00_07  H00_08  H00_09  H00_10  H00_11  H00_12  \
999       98     1.0  70348.0     4.0    24.0    10.0  2020.0    15.0    40.0   
1000      98     1.0  70364.0     4.0    21.0    10.0  2020.0    15.0    36.0   
1001      98     1.0  70362.0     4.0    25.0    10.0  2020.0    12.0    30.0   
1002      98     1.0  70465.0     4.0    24.0    10.0  2020.0    17.0    35.0   
1003      98     1.0  70577.0     4.0     1.0    11.0  2020.0    16.0     0.0   
...      ...     ...      ...     ...     ...     ...     ...     ...     ...   
52564     98     1.0  70390.0     1.0     1.0    11.0  2020.0    12.0    22.0   
52565     98     1.0  70506.0     1.0    31.0    10.0  2020.0    14.0    55.0   
52566     98     1.0  70389.0     1.0    31.0    10.0  2020.0    16.0     8.0   
52567     98     1.0  70602.0     1.0     1.0    11.0  2020.0    13.0    12.0   
52568     98     1.0  70635.0     1.0     1.0    11.0  2020.0    15.0     1.0   

       H00_13  ...  PM_

      Якутия городская местность          
                            Абс.      Отн.
Нет                          219  0.737374
Да                            56  0.188552
None                          22  0.074074
Total                        297  1.000000
-------------------------------------------------------------------
Наличие мусоропровода


      Якутия городская местность          
                            Абс.      Отн.
Нет                          220  0.740741
Да                            55  0.185185
None                          22  0.074074
Total                        297  1.000000
-------------------------------------------------------------------
Вид водопровода в доме


                            Якутия городская местность          
                                                  Абс.      Отн.
Централизованный водопровод                        273  0.919192
Водопровод отсутствует                              23  0.077441
Водопровод из колодца                                1  0.003367
Total                                              297  1.000000
-------------------------------------------------------------------
Наличие стиральной машины


                                         Якутия городская местность          
                                                               Абс.      Отн.
Да и имеется и используется                                     292  0.983165
Нет, не имеется, не можем себе позволить                          3  0.010101
Нет, не имею по другим причинам                                   2  0.006734
Total                                                           297  1.000000
-------------------------------------------------------------------
Наличие холодильника


...

               Якутия сельская местность          
                                    Абс.      Отн.
None                                 180  0.952381
шум от соседей                         9  0.047619
Total                                189  1.000000
-------------------------------------------------------------------
Нарушение общественного порядка вблизи дома или в подъезде


                                                   Якутия сельская местность  \
                                                                        Абс.   
None                                                                     186   
нарушение общественного порядка вблизи дома или...                         3   
Total                                                                    189   

                                                              
                                                        Отн.  
None                                                0.984127  
нарушение общественного порядка вблизи дома или...  0.015873  
Total                                               1.000000  
-------------------------------------------------------------------
Оценка состояния жилого помещения


                   Якутия сельская местность          
                                        Абс.      Отн.
Удовлетворительное                        88  0.465608
Хорошее                                   62  0.328042
Отличное                                  18  0.095238
Плохое                                    16  0.084656
Очень плохое                               5  0.026455
Total                                    189  1.000000
-------------------------------------------------------------------
Условия проживания домохозяйства


                                             Якутия сельская местность  \
                                                                  Абс.   
Индивидуальный (одноквартирный) дом                                160   
Отдельная квартира                                                  21   
Часть индивидуального (одноквартирного) дома                         8   
Total                                                              189   

                                                        
                                                  Отн.  
Индивидуальный (одноквартирный) дом           0.846561  
Отдельная квартира                            0.111111  
Часть индивидуального (одноквартирного) дома  0.042328  
Total                                         1.000000  
-------------------------------------------------------------------
Является ли основное жилье собственностью домохозяйства


                                                   Якутия сельская местность  \
                                                                        Абс.   
Да, без использования ипотечных средств или ипо...                       140   
Нет                                                                       44   
Да, с использованием ипотечных средств (ипотека...                         5   
Total                                                                    189   

                                                              
                                                        Отн.  
Да, без использования ипотечных средств или ипо...  0.740741  
Нет                                                 0.232804  
Да, с использованием ипотечных средств (ипотека...  0.026455  
Total                                               1.000000  
-------------------------------------------------------------------


In [21]:
# выбираем тип города
for name in yakutia['R_1_1'].unique():
    selected = yakutia[yakutia['R_1_1']==name]
    name = meta.value_labels[meta.variable_to_label['R_1_1']].get(name)
    
    frequencuies_pandas(selected, columns_to_count_frequencies, factors, f"Якутия {name}")
    print('===================================================================')

Наличие лифта


      Якутия городской,  250,0 - 499,9 тыс.чел          
                                          Абс.      Отн.
Нет                                         53  0.654321
Да                                          28  0.345679
Total                                       81  1.000000
-------------------------------------------------------------------
Наличие мусоропровода


      Якутия городской,  250,0 - 499,9 тыс.чел     
                                          Абс. Отн.
Нет                                         81  1.0
Total                                       81  1.0
-------------------------------------------------------------------
Вид водопровода в доме


                            Якутия городской,  250,0 - 499,9 тыс.чел     
                                                                Абс. Отн.
Централизованный водопровод                                       81  1.0
Total                                                             81  1.0
-------------------------------------------------------------------
Наличие стиральной машины


                                Якутия городской,  250,0 - 499,9 тыс.чел  \
                                                                    Абс.   
Да и имеется и используется                                           79   
Нет, не имею по другим причинам                                        2   
Total                                                                 81   

                                           
                                     Отн.  
Да и имеется и используется      0.975309  
Нет, не имею по другим причинам  0.024691  
Total                            1.000000  
-------------------------------------------------------------------
Наличие холодильника


...

      Якутия сельский,  более 5000 чел.     
                                   Абс. Отн.
None                                 27  1.0
Total                                27  1.0
-------------------------------------------------------------------
Нарушение общественного порядка вблизи дома или в подъезде


      Якутия сельский,  более 5000 чел.     
                                   Абс. Отн.
None                                 27  1.0
Total                                27  1.0
-------------------------------------------------------------------
Оценка состояния жилого помещения


                   Якутия сельский,  более 5000 чел.          
                                                Абс.      Отн.
Хорошее                                           15  0.555556
Удовлетворительное                                 9  0.333333
Отличное                                           3  0.111111
Total                                             27  1.000000
-------------------------------------------------------------------
Условия проживания домохозяйства


                                    Якутия сельский,  более 5000 чел.     
                                                                 Абс. Отн.
Индивидуальный (одноквартирный) дом                                27  1.0
Total                                                              27  1.0
-------------------------------------------------------------------
Является ли основное жилье собственностью домохозяйства


                                                   Якутия сельский,  более 5000 чел.  \
                                                                                Абс.   
Да, без использования ипотечных средств или ипо...                                24   
Нет                                                                                2   
Да, с использованием ипотечных средств (ипотека...                                 1   
Total                                                                             27   

                                                              
                                                        Отн.  
Да, без использования ипотечных средств или ипо...  0.888889  
Нет                                                 0.074074  
Да, с использованием ипотечных средств (ипотека...  0.037037  
Total                                               1.000000  
-------------------------------------------------------------------


In [ ]:
for k, v in factors.items():
    split_question = pd.concat(v, axis=1).iloc[:,:12]
    # папку tables надо создать на диске заранее, ну или поискать, как создавать папки на диске отсюда из колаба
    # split_question.to_excel(f'drive/MyDrive/tables/{k}.xlsx')
    print(k)
    
    print("Сельская и городская местность, вся РФ")

    # Критерий существенности различия
    omegas_1 = split_question['сельская местность','Отн.'][:-1]
    omegas_2 = split_question['городская местность', 'Отн.'][:-1]
    # в этих данных порядки n это десятки тысяч, и даже различия частостей на тысячные доли могут оказаться значимыми
    # можно попробовать поставить n меньше, например, 100, и увидеть, что значимых различий становится меньше 
    n1 = split_question['сельская местность','Абс.']['Total']
    n2 = split_question['городская местность', 'Абс.']['Total']
    for response, o1, o2 in zip(split_question[:-1].index, omegas_1, omegas_2):
        print(response)
        print(o1, o2)
        print(n1, n2)
        criterion = abs(o1-o2)/math.sqrt((o1*(1-o1)/n1 + o2*(1-o2)/n2))
        print(f"Критерий существенности различия: {criterion}")
        print('------------------------------------------------------')

Наличие лифта
Сельская и городская местность, вся РФ
Нет
0.2886614684860299 0.5118720801425612
18468 41526
Критерий существенности различия: 53.92220278883004
------------------------------------------------------
None
0.7058696123023609 0.18949573761017194
18468 41526
Критерий существенности различия: 133.59193082967084
------------------------------------------------------
Да
0.00546891921160927 0.2986321822472668
18468 41526
Критерий существенности различия: 126.88361228032454
------------------------------------------------------
Наличие мусоропровода
Сельская и городская местность, вся РФ
Нет
0.2883365821962313 0.5472715888840727
18468 41526
Критерий существенности различия: 62.658214623932814
------------------------------------------------------
None
0.7058696123023609 0.18949573761017194
18468 41526
Критерий существенности различия: 133.59193082967084
------------------------------------------------------
Да
0.0057938055014078406 0.2632326735057554
18468 41526
Критерий существе

In [ ]:
for k, v in factors.items():
    split_question = pd.concat(v, axis=1).iloc[:,:12]
    # папку tables надо создать на диске заранее, ну или поискать, как создавать папки на диске отсюда из колаба
    # split_question.to_excel(f'drive/MyDrive/tables/{k}.xlsx')
    print(k)
    
    print("Сельская и городская местность, Якутия")

    # Критерий существенности различия
    omegas_1 = split_question['Якутия сельская местность','Отн.'][:-1]
    omegas_2 = split_question['Якутия городская местность', 'Отн.'][:-1]
    # в этих данных порядки n это десятки тысяч, и даже различия частостей на тысячные доли могут оказаться значимыми
    # можно попробовать поставить n меньше, например, 100, и увидеть, что значимых различий становится меньше 
    n1 = split_question['Якутия сельская местность','Абс.']['Total']
    n2 = split_question['Якутия городская местность', 'Абс.']['Total']
    for response, o1, o2 in zip(split_question[:-1].index, omegas_1, omegas_2):
        print(response)
        print(o1, o2)
        print(n1, n2)
        criterion = abs(o1-o2)/math.sqrt((o1*(1-o1)/n1 + o2*(1-o2)/n2))
        print(f"Критерий существенности различия: {criterion}")
        print('------------------------------------------------------')

Наличие лифта
Сельская и городская местность, Якутия
Нет
0.1111111111111111 0.7373737373737373
189.0 297
Критерий существенности различия: 18.273060445080933
------------------------------------------------------
None
0.8888888888888888 0.07407407407407407
189.0 297
Критерий существенности различия: 29.68362070239246
------------------------------------------------------
Да
nan 0.18855218855218855
189.0 297
Критерий существенности различия: nan
------------------------------------------------------
Наличие мусоропровода
Сельская и городская местность, Якутия
Нет
0.1111111111111111 0.7407407407407407
189.0 297
Критерий существенности различия: 18.413837543883965
------------------------------------------------------
None
0.8888888888888888 0.07407407407407407
189.0 297
Критерий существенности различия: 29.68362070239246
------------------------------------------------------
Да
nan 0.18518518518518517
189.0 297
Критерий существенности различия: nan
---------------------------------------

In [ ]:
for k, v in factors.items():
    split_question = pd.concat(v, axis=1).iloc[:,:12]

    print(k)
    # чтобы тест был применим, должна быть одинаковая сумма наблюдений в обоих рядах
    # поэтому берем нормализованные значения, а затем умножаем на 1000, чтобы числа не были маленькие
    print("Хихи (извените) квадрат, Вся РФ и городская местность вся РФ")
    chi = chisquare(split_question['городская местность', 'Отн.'][:-1].apply(lambda x: x*1000), f_exp=split_question['Вся РФ', 'Отн.'][:-1].apply(lambda x: x*1000), ddof=0)
    print(chi)
    #print(chi.statistic * split_question['городская местность', 'Абс.']['Total'])
    print(f"Количество степеней свободы {split_question['городская местность', 'Отн.'][:-1].shape[0] - 1}") # количество ответов на вопрос - 1
    print("Хихи (извените) квадрат, вся РФ и сельская местность вся РФ")
    chi = chisquare(split_question['сельская местность', 'Отн.'][:-1].apply(lambda x: x*1000), f_exp=split_question['Вся РФ', 'Отн.'][:-1].apply(lambda x: x*1000), ddof=0, axis=0)
    print(chi)
    #print(chi.statistic * split_question['сельская местность', 'Абс.']['Total'])


    print('-----------------------------------------------------------')

Наличие лифта
Хихи (извените) квадрат, Вся РФ и городская местность вся РФ
Power_divergenceResult(statistic=122.24708802042366, pvalue=2.8469649602834148e-27)
Количество степеней свободы 2
Хихи (извените) квадрат, вся РФ и сельская местность вся РФ
Power_divergenceResult(statistic=618.0718872374902, pvalue=6.12907557684304e-135)
-----------------------------------------------------------
Наличие мусоропровода
Хихи (извените) квадрат, Вся РФ и городская местность вся РФ
Power_divergenceResult(statistic=120.2345650048054, pvalue=7.787462030757597e-27)
Количество степеней свободы 2
Хихи (извените) квадрат, вся РФ и сельская местность вся РФ
Power_divergenceResult(statistic=607.8967254523338, pvalue=9.928949103974199e-133)
-----------------------------------------------------------
Вид водопровода в доме
Хихи (извените) квадрат, Вся РФ и городская местность вся РФ
Power_divergenceResult(statistic=47.005988609364024, pvalue=1.5205420131047836e-09)
Количество степеней свободы 4
Хихи (извенит